<a href="https://colab.research.google.com/github/MoMus2000/Image-classification/blob/master/Beauty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"momus01","key":"10fcceedff5c6b953c23e55dd5fd7078"}'}

In [2]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d gpiosenka/beauty-detection-data-set

 99% 167M/168M [00:01<00:00, 70.2MB/s]
100% 168M/168M [00:01<00:00, 100MB/s] 


In [5]:
!unzip /content/beauty-detection-data-set.zip

Streaming output truncated to the last 5000 lines.
  inflating: consolidated/beautiful/1915.jpg  
  inflating: consolidated/beautiful/1916.jpg  
  inflating: consolidated/beautiful/1917.jpg  
  inflating: consolidated/beautiful/1918.jpg  
  inflating: consolidated/beautiful/1919.jpg  
  inflating: consolidated/beautiful/1920.jpg  
  inflating: consolidated/beautiful/1921.jpg  
  inflating: consolidated/beautiful/1922.jpg  
  inflating: consolidated/beautiful/1923.jpg  
  inflating: consolidated/beautiful/1924.jpg  
  inflating: consolidated/beautiful/1925.jpg  
  inflating: consolidated/beautiful/1926.jpg  
  inflating: consolidated/beautiful/1927.jpg  
  inflating: consolidated/beautiful/1928.jpg  
  inflating: consolidated/beautiful/1929.jpg  
  inflating: consolidated/beautiful/1930.jpg  
  inflating: consolidated/beautiful/1931.jpg  
  inflating: consolidated/beautiful/1932.jpg  
  inflating: consolidated/beautiful/1933.jpg  
  inflating: consolidated/beautiful/1934.jpg  
  inflati

In [30]:
import tensorflow as tf

In [31]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Dropout,BatchNormalization


In [36]:
train_gen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input, horizontal_flip=True).flow_from_directory(
        '/content/train',  target_size=(300, 300), 
        batch_size=128, class_mode='categorical', color_mode='rgb')


Found 4000 images belonging to 2 classes.


In [37]:
test_gen=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input, horizontal_flip=True).flow_from_directory(
        '/content/test',  target_size=(300, 300), 
        batch_size=128, class_mode='categorical', color_mode='rgb')


Found 300 images belonging to 2 classes.


In [38]:
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import  Adamax

mobile = tf.keras.applications.mobilenet.MobileNet( include_top=False,
                                                               input_shape=(300, 300,3),
                                                               pooling='max', weights='imagenet',
                                                               alpha=1, depth_multiplier=1,dropout=.5)                                                          
x=mobile.layers[-1].output
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
predictions=Dense (2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)    
for layer in model.layers:
    layer.trainable=True
model.compile(Adamax(lr=.1), loss='categorical_crossentropy', metrics=['accuracy'])   

In [39]:
model.fit(train_gen,epochs=30,validation_data=test_gen,steps_per_epoch=len(train_generator))

Epoch 1/30
32/32 [==============================] - 43s 1s/step - loss: 4.2178 - accuracy: 0.6885 - val_loss: 7.2096 - val_accuracy: 0.5000
Epoch 2/30
32/32 [==============================] - 40s 1s/step - loss: 0.5252 - accuracy: 0.8615 - val_loss: 5.0975 - val_accuracy: 0.5000
Epoch 3/30
32/32 [==============================] - 40s 1s/step - loss: 0.3807 - accuracy: 0.9183 - val_loss: 3.8308 - val_accuracy: 0.5333
Epoch 4/30
32/32 [==============================] - 41s 1s/step - loss: 0.2276 - accuracy: 0.9513 - val_loss: 2.7302 - val_accuracy: 0.5233
Epoch 5/30
32/32 [==============================] - 41s 1s/step - loss: 0.1002 - accuracy: 0.9700 - val_loss: 2.1964 - val_accuracy: 0.5633
Epoch 6/30
32/32 [==============================] - 41s 1s/step - loss: 0.1146 - accuracy: 0.9675 - val_loss: 1.5386 - val_accuracy: 0.6833
Epoch 7/30
32/32 [==============================] - 41s 1s/step - loss: 0.1156 - accuracy: 0.9710 - val_loss: 1.2663 - val_accuracy: 0.7367
Epoch 8/30
32/32 [==

In [61]:
import cv2
img = cv2.imread('/content/mama.jpg')
img = cv2.resize(img,(300,300))

In [62]:
img = np.reshape(img,[-1,300,300,3])
img = img/255.0

In [63]:
prediction = ((model.predict(img)))

In [64]:
print(np.argmax(prediction))

1


In [47]:
print((train_gen.class_indices))

{'average': 0, 'beautiful': 1}
